# 预测

In [34]:
import torch

model = torch.load("../nn/nn_models/nn_model_epoch_29.pkl")
model


LSTM_attention(
  (embedding): Embedding(54848, 50)
  (encoder): LSTM(50, 100, num_layers=6, batch_first=True, dropout=0.2, bidirectional=True)
  (decoder1): Linear(in_features=200, out_features=100, bias=True)
  (decoder2): Linear(in_features=100, out_features=2, bias=True)
)

# 可视化

In [35]:
def read_word2id(file_path):
    word_to_idx = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:
                word, idx = line.split()
                word_to_idx[word] = int(idx)
    return word_to_idx

# 使用示例：
word_to_idx = read_word2id('../word2vec_data/word2id.txt')


In [36]:
import torch
import jieba

def classify_review(comment, model):
    words = list(jieba.cut(comment, cut_all=True))
    word_indices = [word_to_idx[word] if word in word_to_idx else word_to_idx['_PAD_'] for word in words]

    # 将词索引转换为张量
    words_tensor = torch.tensor(word_indices).unsqueeze(0)
    # print("words_tensor: ", words_tensor)
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        model.to(device)
        words_tensor = words_tensor.to(device)

    # 使用模型进行分类
    with torch.no_grad():
        embeddings = model.embedding(words_tensor)  # [batch, seq_len] => [batch, seq_len, embed_dim]
        states, hidden = model.encoder(embeddings)  # [batch, seq_len, embed_dim]
        output = model.decoder1(states[:, -1, :])
        output = model.decoder2(output)
        # print("output:", output)
        _, pred = torch.max(output, 1)
        print("nn_prediction: ", pred)

    if pred.item() == 0:
        print("好评")
        return 0
    elif pred.item() == 1:
        print("坏评")
        return 1

In [37]:
# 示例用法
comment = input("请输入你的评论：")
print(comment)
classification = classify_review(comment, model)
print("评论分类：", classification)


这个真不错！
nn_prediction:  tensor([0], device='cuda:0')
好评
评论分类： 0
